<img height="60px" src="https://colab.research.google.com/img/colab_favicon.ico" align="left" hspace="20px" vspace="5px">

# Building a text classification model with TF Hub

In this notebook, we'll walk you through building a model to predict the genres of a movie given its description. The emphasis here is not on accuracy, but instead how to use TF Hub layers in a text classification model.

To start, import the necessary dependencies for this project.

* References:

[https://medium.com/tensorflow/building-a-text-classification-model-with-tensorflow-hub-and-estimators-3169e7aa568](https://medium.com/tensorflow/building-a-text-classification-model-with-tensorflow-hub-and-estimators-3169e7aa568)

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/learning-stack/Colab-ML-Playbook/blob/master/NLP/NLP%20-%20Text%20Classification%20Model%20With%20Tensorflow%20Hub/movie_descriptions_tfhub.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/learning-stack/Colab-ML-Playbook/blob/master/NLP/NLP%20-%20Text%20Classification%20Model%20With%20Tensorflow%20Hub/movie_descriptions_tfhub.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In [0]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import json
import pickle
import urllib

from sklearn.preprocessing import MultiLabelBinarizer

print(tf.__version__)

1.10.1


## The dataset

We need a lot of text inputs to train our model. For this model we'll use [this awesome movies dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset) from Kaggle. To simplify things I've made the `movies_metadata.csv` file available in a public Cloud Storage bucket so we can download it with `wget`. I've preprocessed the dataset already to limit the number of genres we'll use for our model, but first let's take a look at the original data so we can see what we're working with.

In [0]:
# Download the data from GCS
!wget 'https://storage.googleapis.com/movies_data/movies_metadata.csv'


Redirecting output to ‘wget-log’.


Next we'll convert the dataset to a Pandas dataframe and print the first 5 rows. For this model we're only using 2 of these columns: `genres` and `overview`.

In [0]:
data = pd.read_csv('movies_metadata.csv')
data.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


## Preparing the data for our model

I've done some preprocessing to limit the dataset to the top 9 genres, and I've saved the Pandas dataframes as public [Pickle](https://docs.python.org/3/library/pickle.html) files in GCS. Here we download those files. The resulting `descriptions` and `genres` variables are Pandas Series containing all descriptions and genres from our dataset respectively.

In [0]:
urllib.request.urlretrieve('https://storage.googleapis.com/bq-imports/descriptions.p', 'descriptions.p')
urllib.request.urlretrieve('https://storage.googleapis.com/bq-imports/genres.p', 'genres.p')

descriptions = pickle.load(open('descriptions.p', 'rb'))
genres = pickle.load(open('genres.p', 'rb'))

In [0]:
print (genres)

2                           [Romance, Comedy]
4                                    [Comedy]
6                           [Comedy, Romance]
8               [Action, Adventure, Thriller]
9               [Adventure, Action, Thriller]
11                           [Comedy, Horror]
14                        [Action, Adventure]
17                            [Crime, Comedy]
18                 [Crime, Comedy, Adventure]
19                    [Action, Comedy, Crime]
20                  [Comedy, Thriller, Crime]
22       [Action, Adventure, Crime, Thriller]
32                       [Romance, Adventure]
51                          [Comedy, Romance]
62                                   [Comedy]
63                          [Comedy, Romance]
64                                   [Comedy]
65                  [Action, Science Fiction]
67                          [Comedy, Romance]
68                                   [Comedy]
69          [Horror, Action, Thriller, Crime]
70                [Action, Thrille

### Splitting our data
When we train our model, we'll use 80% of the data for training and set aside 20% of the data to evaluate how our model performed.

In [0]:
train_size = int(len(descriptions) * .8)

train_descriptions = descriptions[:train_size].astype('str')
train_genres = genres[:train_size]

test_descriptions = descriptions[train_size:].astype('str')
test_genres = genres[train_size:]

### Formatting our labels
When we train our model we'll provide the labels (in this case genres) associated with each movie. We can't pass the genres in as strings directly, we'll transform them into multi-hot vectors. Since we have 9 genres, we'll have a 9 element vector for each movie with 0s and 1s indicating which genres are present in each description.

In [0]:
encoder = MultiLabelBinarizer()
encoder.fit_transform(train_genres)
train_encoded = encoder.transform(train_genres)
test_encoded = encoder.transform(test_genres)
num_classes = len(encoder.classes_)

# Print all possible genres and the labels for the first movie in our training dataset
print(encoder.classes_)
print(train_encoded[0])

['Action' 'Adventure' 'Comedy' 'Crime' 'Documentary' 'Horror' 'Romance'
 'Science Fiction' 'Thriller']
[0 0 1 0 0 0 1 0 0]


### Create our TF Hub embedding layer
Our model will only have one feature (the description) and it’ll be represented as an embedding column. Text embeddings provide a way to represent pieces of text in vector space, so that similar words or sentences are closer together in the embedding space. TF Hub simplifies this process by providing text embeddings that have already been trained on a variety of text data.


[TF Hub]() provides a library of existing pre-trained model checkpoints for various kinds of models (images, text, and more) In this model we'll use the TF Hub `universal-sentence-encoder` module for our pre-trained word embeddings. **Universal sentence encoder is famous for longer form text inputs**. We only need one line of code to instantiate module. When we train our model, it'll convert our array of movie description strings to embeddings. When we train our model, we'll use this as a feature column.


In [0]:
description_embeddings = hub.text_embedding_column("descriptions", module_spec="https://tfhub.dev/google/universal-sentence-encoder/2", trainable=False)


INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/2'.
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/2'.


## Instantiating our DNNEstimator Model

Since each label needs to be the same length, we’ll transform these lists into multi-hot vectors of 1s and 0s corresponding to the genres present in a particular description.


The first parameter we pass to our DNNEstimator is called a head, and defines the type of labels our model should expect. Since we want our model to output multiple labels, we’ll use multi_label_head here. Then we'll convert our features and labels to numpy arrays and instantiate our Estimator. `batch_size` and `num_epochs` are hyperparameters - you should experiment with different values to see what works best on your dataset.


To transform our string labels into multi-hot vectors in just a few lines of code we’ll use a scikit learn utility called MultiLabelBinarizer.

In [0]:
multi_label_head = tf.contrib.estimator.multi_label_head(
    num_classes,
    loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE
)

The hidden_units param indicates how many layers we’ll have in our network. This model has 2 layers, the first has 64 neurons and the second has 10. 

In [0]:
features = {
  "descriptions": np.array(train_descriptions).astype(np.str)
}
labels = np.array(train_encoded).astype(np.int32)
train_input_fn = tf.estimator.inputs.numpy_input_fn(features, labels, shuffle=True, batch_size=32, num_epochs=25)
estimator = tf.contrib.estimator.DNNEstimator(
    head=multi_label_head,
    hidden_units=[64,10],
    feature_columns=[description_embeddings])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp3ybvbz_d', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6aac46d550>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Training and serving our model 
To train our model, we simply call `train()` passing it the input function we defined above. Once our model is trained, we'll define an evaluation input function similar to the one above and call `evaluate()`. When this completes we'll get a few metrics we can use to evaluate our model's accuracy.


In [0]:
estimator.train(input_fn=train_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp3ybvbz_d/model.ckpt.
INFO:tensorflow:loss = 0.693498, step = 1
INFO:tensorflow:global_step/sec: 47.6126
INFO:tensorflow:loss = 0.48309192, step = 101 (2.106 sec)
INFO:tensorflow:global_step/sec: 57.0742
INFO:tensorflow:loss = 0.44566014, step = 201 (1.752 sec)
INFO:tensorflow:global_step/sec: 56.3147
INFO:tensorflow:loss = 0.43010002, step = 301 (1.772 sec)
INFO:tensorflow:global_step/sec: 55.8207
INFO:tensorflow:loss = 0.39764452, step = 401 (1.791 sec)
INFO:tensorflow:global_step/sec: 57.1101
INFO:tensorflow:loss = 0.390834

In [0]:
# Define our eval input_fn and run eval
eval_input_fn = tf.estimator.inputs.numpy_input_fn({"descriptions": np.array(test_descriptions).astype(np.str)}, test_encoded.astype(np.int32), shuffle=False)
estimator.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-10-15:28:22
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp3ybvbz_d/model.ckpt-9371
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-10-15:28:33
INFO:tensorflow:Saving dict for global step 9371: auc = 0.91783077, auc_precision_recall = 0.749413, average_loss = 0.24026607, global_step = 9371, loss = 0.24114418
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 9371: /tmp/tmp3ybvbz_d/model.ckpt-9371


{'auc': 0.91783077,
 'auc_precision_recall': 0.749413,
 'average_loss': 0.24026607,
 'global_step': 9371,
 'loss': 0.24114418}

This model achieved **91.7%** AUC, and** 75%** precision / recall.

## Generating predictions on new data
Now for the most fun part! Let's generate predictions on movie descriptions our model hasn't seen before. We'll define an array of 3 new description strings (the comments indicate the correct genres) and create a `predict_input_fn`. Then we'll display the top 2 genres along with their confidence percentages for each of the 3 movies.

In [0]:
# Test our model on some raw description data
raw_test = [
    "An examination of our dietary choices and the food we put in our bodies. Based on Jonathan Safran Foer's memoir.", # Documentary
    "After escaping an attack by what he claims was a 70-foot shark, Jonas Taylor must confront his fears to save those trapped in a sunken submersible.", # Action, Adventure
    "A teenager tries to survive the last week of her disastrous eighth-grade year before leaving to start high school.", # Comedy
]



In [0]:
# Generate predictions
predict_input_fn = tf.estimator.inputs.numpy_input_fn({"descriptions": np.array(raw_test).astype(np.str)}, shuffle=False)
results = estimator.predict(predict_input_fn)

In [0]:
# Display predictions
for movie_genres in results:
  top_2 = movie_genres['probabilities'].argsort()[-2:][::-1]
  for genre in top_2:
    text_genre = encoder.classes_[genre]
    print(text_genre + ': ' + str(round(movie_genres['probabilities'][genre] * 100, 2)) + '%')
  print('')

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp3ybvbz_d/model.ckpt-9371
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Documentary: 98.42%
Comedy: 15.77%

Horror: 88.09%
Thriller: 39.53%

Comedy: 64.15%
Horror: 27.68%

